In [ ]:
from google.colab import drive
drive.mount('/content//gdrive')

Mounted at /content//gdrive


In [ ]:
# Courtesy of Kaggle user 'NoBugs'
# Transforms string of numbers in .csv to image and saves it

import numpy as np
import cv2
import pandas as pd
import random
import os

curdir = "/content/"
#               0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
#class_names = ['afraid', 'angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']
def gen_record(csvfile,channel):
    data = pd.read_csv(csvfile,delimiter=',',dtype='a')
    labels = np.array(data['emotion'],np.float)

    imagebuffer = np.array(data['pixels'])
    images = np.array([np.fromstring(image,np.uint8,sep=' ') for image in imagebuffer])
    del imagebuffer
    num_shape = int(np.sqrt(images.shape[-1]))
    images.shape = (images.shape[0],num_shape,num_shape)
    dirs = set(data['Usage'])
    subdirs = set(labels)
    class_dir = {}
    for dr in dirs:
        dest = os.path.join(curdir,dr)
        class_dir[dr] = dest
        if not os.path.exists(dest):
            os.mkdir(dest)

    data = zip(labels,images,data['Usage'])

    for d in data:
        label = int(d[0])
        if label == 0:
             destdir = os.path.join(class_dir[d[-1]],'angry')
        if label == 1: 
             destdir = os.path.join(class_dir[d[-1]],'disgusted')
        if label == 2:
             destdir = os.path.join(class_dir[d[-1]],'afraid')
        if label == 3: 
             destdir = os.path.join(class_dir[d[-1]],'happy')
        if label == 4: 
             destdir = os.path.join(class_dir[d[-1]],'sad')
        if label == 5: 
             destdir = os.path.join(class_dir[d[-1]],'surprised')
        if label == 6: 
             destdir = os.path.join(class_dir[d[-1]],'neutral')
        

        if not os.path.exists(destdir):
            os.mkdir(destdir)

        img = d[1]
        filepath = unique_name(destdir,d[-1])
        print('[^_^] Write image to %s' % filepath)

        if not filepath:
            continue

        sig = cv2.imwrite(filepath,img)
        if not sig:
            print('Error')
            exit(-1)


def unique_name(pardir,prefix,suffix='jpg'):
    filename = '{0}_{1}.{2}'.format(prefix,random.randint(1,10**8),suffix)
    filepath = os.path.join(pardir,filename)
    if not os.path.exists(filepath):
        return filepath
    unique_name(pardir,prefix,suffix)


filename = 'fer2013.csv'
filename = os.path.join("/content/gdrive/MyDrive/",filename)
gen_record(filename,1)

Streaming output truncated to the last 5000 lines.
[^_^] Write image to /content/PublicTest/angry/PublicTest_11434104.jpg
[^_^] Write image to /content/PublicTest/neutral/PublicTest_16752205.jpg
[^_^] Write image to /content/PublicTest/surprised/PublicTest_23947022.jpg
[^_^] Write image to /content/PublicTest/angry/PublicTest_80865963.jpg
[^_^] Write image to /content/PublicTest/happy/PublicTest_44148922.jpg
[^_^] Write image to /content/PublicTest/sad/PublicTest_57461852.jpg
[^_^] Write image to /content/PublicTest/happy/PublicTest_24413575.jpg
[^_^] Write image to /content/PublicTest/happy/PublicTest_13576334.jpg
[^_^] Write image to /content/PublicTest/afraid/PublicTest_31146294.jpg
[^_^] Write image to /content/PublicTest/afraid/PublicTest_52414714.jpg
[^_^] Write image to /content/PublicTest/happy/PublicTest_53549033.jpg
[^_^] Write image to /content/PublicTest/sad/PublicTest_61122040.jpg
[^_^] Write image to /content/PublicTest/surprised/PublicTest_48174654.jpg
[^_^] Write image 

In [ ]:
# import zipfile
# with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
#     zip_ref.extractall("/content/")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import numpy as np
# from skimage import transform
import cv2
import sys

In [ ]:
# # Names of the integer classes, i.e., 0 -> afraid, 1 -> angry, etc.
class_names = ['afraid', 'angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#     directory=r"/content/archive/DorinNetSorted",
#     target_size=(224, 224),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [ ]:
from keras.models import load_model
import tensorflow_hub as hub
import tensorflow as tf

# mobilenet_model =  load_model('/content/gdrive/MyDrive/MobileNetV2_last_model.h5')
# inceptionV3_model = load_model('/content/gdrive/MyDrive/InceptionV3TLonFacialExpr.h5', custom_objects={'KerasLayer':hub.KerasLayer})
# resnet_model = load_model('/content/gdrive/MyDrive/ResnetV2_bestweights_epochs_008-val_accuracy_0.849.hdf5', custom_objects={'KerasLayer':hub.KerasLayer})

mobilenet_model =  load_model('/content/gdrive/MyDrive/MyCNN/MobileNet_bestweights_epochs_056-val_accuracy_0.845.hdf5', custom_objects={'KerasLayer':hub.KerasLayer})
resnet_model = load_model('/content/gdrive/MyDrive/MyCNN/ResnetV2_bestweights_epochs_048-val_accuracy_0.854 (1).hdf5', custom_objects={'KerasLayer':hub.KerasLayer})
inceptionV3_model = load_model('/content/gdrive/MyDrive/InceptionV3TLonFacialExpr.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)
# this is a similar generator, for validation data
batch_size =32
validation_size = 3533 # size of either test set
test_generator = test_datagen.flow_from_directory('/content/PublicTest',target_size=(224, 224),batch_size = 1, color_mode = 'rgb')

test_batches_resnet = ImageDataGenerator(preprocessing_function= tf.keras.applications.resnet.preprocess_input).flow_from_directory(directory='/content/PublicTest',target_size=(224,224),classes=class_names,batch_size=32,shuffle = False)
test_batches_inception = ImageDataGenerator(preprocessing_function= tf.keras.applications.inception_v3.preprocess_input).flow_from_directory(directory='/content/PublicTest',target_size=(299,299),classes=class_names,batch_size=32,shuffle = False)
test_batches_mobilenet = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(directory='/content/PublicTest',target_size=(224,224),classes=class_names,batch_size=32,shuffle = False)


# ~~~~~~~~~~~~~~~~ Check accuracy & F-score ~~~~~~~~~~~~~~~
# score = resnet_model.evaluate(test_batches_resnet, verbose=2)
# print("TEST")
# print(score)
# print("Loss: {0} \nAccuracy: {1} \nF-Score: {2}").format(score[0], score[1], score[2])


# Re-evaluate the model
loss, acc = resnet_model.evaluate(test_batches_resnet, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# type(test_generator)
# type(test_batches)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i in range (cm.shape[0]):
      for j in range (cm.shape[1]):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
y_pred =  resnet_model.predict(test_batches_resnet, batch_size=10, verbose=0) 
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_batches_resnet.classes, y_pred)

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot  as skplt
skplt.metrics.plot_confusion_matrix(test_batches_resnet.classes, y_pred, normalize=True)
plt.show()


In [ ]:
y_pred =  inceptionV3_model.predict(test_batches_inception, batch_size=10, verbose=0) 
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_batches_inception.classes, y_pred)

In [ ]:
import scikitplot  as skplt
skplt.metrics.plot_confusion_matrix(test_batches_inception.classes, y_pred, normalize=True)
plt.show()

In [ ]:
y_pred =  mobilenet_model.predict(test_batches_inception, batch_size=10, verbose=0) 
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_batches_inception.classes, y_pred)

In [ ]:
cm_plot_labels = class_names
# normalize the data
cm = cm / cm.astype(np.float).sum(axis=1)[:, np.newaxis] # recall  (tp)/(tp+fn) 
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix MobileNet')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = inceptionV3_model.predict(images, batch_size=10)
  print(fn)
  print(classes)

In [ ]:
train_eval_result = model.evaluate(dict(train_df), train_df['Sentiment'])
validation_eval_result = model.evaluate(dict(validation_df), validation_df['Sentiment'])

print(f"Training set accuracy: {train_eval_result[1]}")
print(f"Validation set accuracy: {validation_eval_result[1]}")

## Confusion matrix

Another very interesting statistic, especially for multiclass problems, is the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). The confusion matrix allows visualization of the proportion of correctly and incorrectly labelled examples. We can easily see how much our classifier is biased and whether the distribution of labels makes sense. Ideally the largest fraction of predictions should be distributed along the diagonal.

In [ ]:
predictions = model.predict(dict(validation_df))
predictions = tf.argmax(predictions, axis=-1)
predictions

In [ ]:
cm = tf.math.confusion_matrix(validation_df['Sentiment'], predictions)
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

In [ ]:
sns.heatmap(
    cm, annot=True,
    xticklabels=SENTIMENT_LABELS,
    yticklabels=SENTIMENT_LABELS)
plt.xlabel("Predicted")
plt.ylabel("True")